<a href="https://colab.research.google.com/github/fhdbbk/fastai-1/blob/main/Understanding_Delegates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
import inspect
import time

In [2]:
class WebPage:
    def __init__(self, title, category="General", date=None, author="Fahad"):
        self.title,self.category,self.author = title,category,author
        self.date = date or datetime.now()

    def other_method(self):
      pass

In [3]:
class ProductPage(WebPage):
    def __init__(self, title, price, cost, category="General", date=None, author="Fahad"):
        super().__init__(title, category=category, date=date, author=author)

class ProductPage2(WebPage):
    def __init__(self, title, price, cost, **kwargs):
        super().__init__(title, **kwargs)

In [ ]:
dir(WebPage)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'other_method']

In [4]:
p1 = ProductPage("Novel", 225.0, 220.0, category='Book', author='Fariya')
p2 = ProductPage("AI", 550.0, 500.0, category='Book', author='Fahad')
print(p1.author)
print(p2.author)

Fariya
Fahad


In [ ]:
dir(p1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'author',
 'category',
 'date',
 'other_method',
 'title']

In [ ]:
p1.__dict__

{'author': 'Fariya',
 'category': 'Book',
 'date': datetime.datetime(2021, 6, 3, 9, 14, 23, 91811),
 'title': 'Novel'}

In [ ]:
sig = inspect.signature(ProductPage)
print(sig)
print(type(sig))
sig.parameters

(title, price, cost, category='General', date=None, author='Fahad')
<class 'inspect.Signature'>


mappingproxy({'author': <Parameter "author='Fahad'">,
              'category': <Parameter "category='General'">,
              'cost': <Parameter "cost">,
              'date': <Parameter "date=None">,
              'price': <Parameter "price">,
              'title': <Parameter "title">})

In [5]:
print(inspect.signature(ProductPage))
print(inspect.signature(ProductPage2))

(title, price, cost, category='General', date=None, author='Fahad')
(title, price, cost, **kwargs)


## Example of a decorator

In [3]:
def show_time(f):
  def inner(*args, **kwargs):
    start_time = time.time()
    res = f(*args, **kwargs)
    time_taken = time.time() - start_time
    print("Time taken to run: {}".format(time_taken))
    return res
  return inner

@show_time
def add(a, b):
  c = a + b
  time.sleep(1)
  return c

# add = show_time(add)
c = add(2, 4)
print(c)

Time taken to run: 1.001905918121338
6


## Decorator with argument

In [4]:
def prefix_decorator(prefix):
  def decorator_func(original_function):
    def wrapper(*args, **kwargs):
      print(prefix, "Executed this before {}".format(original_function.__name__))
      res = original_function(*args, **kwargs)
      print(prefix, "Executed this after {}".format(original_function.__name__))
      return res
    return wrapper
  return decorator_func

@prefix_decorator('LOG:')
def display_info(name, age):
  print("Ran with {} and {}".format(name, age))

display_info("Fahad", 30)

LOG: Executed this before display_info
Ran with Fahad and 30
LOG: Executed this after display_info


In [22]:
def delegates(to=None, keep=False):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to,f
        print(f"to_f: {to_f}\t from_f: {from_f}")
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        print(sigd)
        k = sigd.pop('kwargs')
        print(k)
        print(inspect.signature(to_f).parameters.items())
        for x,v in inspect.signature(to_f).parameters.items():
          print("Looking default for {}".format(x))
          print(v.default)
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd}
        print(s2)
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

In [27]:
@delegates()
class ProductPage2(WebPage):
    def __init__(self, title, price, cost, **kwargs):
        super().__init__(title, **kwargs)

to_f: <function WebPage.__init__ at 0x7fade2ce8050>	 from_f: <function ProductPage2.__init__ at 0x7fade2b67950>
{'self': <Parameter "self">, 'title': <Parameter "title">, 'price': <Parameter "price">, 'cost': <Parameter "cost">, 'kwargs': <Parameter "**kwargs">}
**kwargs
odict_items([('self', <Parameter "self">), ('title', <Parameter "title">), ('category', <Parameter "category='General'">), ('date', <Parameter "date=None">), ('author', <Parameter "author='Fahad'">)])
Looking default for self
<class 'inspect._empty'>
Looking default for title
<class 'inspect._empty'>
Looking default for category
General
Looking default for date
None
Looking default for author
Fahad
{'category': <Parameter "category='General'">, 'date': <Parameter "date=None">, 'author': <Parameter "author='Fahad'">}


In [28]:
print(inspect.signature(ProductPage2))

(title, price, cost, category='General', date=None, author='Fahad')


## Delegated Composition

In [46]:
def custom_dir(c, add): return dir(type(c)) + list(c.__dict__.keys()) + add

class GetAttr:
    "Base class for attr accesses in `self._xtra` passed down to `self.default`"
    @property
    def _xtra(self): return [o for o in dir(self.default) if not o.startswith('_')]
    def __getattr__(self,k):
        if k in self._xtra: return getattr(self.default, k)
        raise AttributeError(k)
    def __dir__(self): return custom_dir(self, self._xtra)

In [47]:
class ProductPage3(GetAttr):
    def __init__(self, page, price, cost):
        self.page,self.price,self.cost = page,price,cost
        self.default = page

In [48]:
page = WebPage('Soap', category='Bathroom', author="Sylvain")
p = ProductPage3(page, 15.0, 10.50)
p.author

'Sylvain'

In [49]:
print(inspect.signature(ProductPage3))

(page, price, cost)


In [50]:
params = [o for o in dir(p) if not o.startswith('_')]

print(params)

['author', 'category', 'cost', 'date', 'default', 'other_method', 'page', 'price', 'title']
